# Build custom image locally

In [ ]:
from google.cloud import aiplatform

REGION = "europe-west3"
PROJECT_ID = "wealthapi-ai"
MODEL_DISPLAY_NAME = "transaction_classification_cpr"
REPOSITORY = "wealthapi"
IMAGE = "transaction-classification-custom-predictor"
GCS_PATH_TO_MODEL_ARTIFACTS = "gs://transaction_classification_frankfurt/model_v1/"

aiplatform.init(project=PROJECT_ID,
                location=REGION)

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel

from src.predictor import CprPredictor

local_model = LocalModel.build_cpr_model(
    "src",
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CprPredictor,
    requirements_path=os.path.join("src", "requirements.txt"),
)

# Check build image

In [ ]:
local_model.get_serving_container_spec()

# Create container locally and test prediction

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=GCS_PATH_TO_MODEL_ARTIFACTS
) as local_endpoint:
    health_check_response = local_endpoint.run_health_check()
    predict_response = local_endpoint.predict(
        request_file="instances.json",
        headers={"Content-Type": "application/json"},
    )

In [ ]:
import codecs

print(health_check_response)
print(predict_response)
print(codecs.decode(predict_response.content, 'unicode_escape'))

In [ ]:
local_model.push_image()

In [ ]:
from google.cloud import aiplatform

model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=GCS_PATH_TO_MODEL_ARTIFACTS,
)